# Basketball NEAR Results Analysis

This notebook analyzes and compares results from:
- ~Baseline NEAR experiments (from baseline_results.json)~
- Neurosym-lib NEAR experiments (from various results pickle files)
- Reported program from the NEAR paper

## Step 0: Data collection.

Compute results for all neurosym-lib experiments and the reported program
```bash
# Search for the top 40 programs on the Fly-v-Fly dataset
$ CUDA_VISIBLE_DEVICES=0 python notebooks/bball_reproduction/benchmark_bball.py

In [ ]:
%load_ext autoreload
%autoreload 2
%cd /home/asehgal/neurosym-lib

import json
import pickle
from pathlib import Path

import numpy as np
import pandas as pd

from neurosym.examples.near.metrics import compute_metrics

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/asehgal/neurosym-lib


## Load Baseline Results

In [ ]:
# baseline results not available as NEAR code
# doesn't include execution code for this.

# # Load best NEAR results
# near_results_pth = Path("outputs/bball_results/near_outputs/")

# baseline_results = {}
# for result_file in near_results_pth.rglob("test_results.json"):
#     print(f"Loading Baseline results from {result_file}")
#     with open(result_file, "r") as f:
#         expt_report = json.load(f)
#         experiment_name = result_file.parent.name
#         y_true = np.array(expt_report["true_vals"])
#         y_scores = np.array(expt_report["predicted_vals"])
#         expt_report["report"] = compute_metrics(y_scores, y_true)
#         baseline_results[experiment_name] = expt_report

Loading Baseline results from outputs/mice_results/near_outputs/crim13_iddfs-near_010/test_results.json
Loading Baseline results from outputs/mice_results/near_outputs/crim13_astar-near_020/test_results.json
Loading Baseline results from outputs/mice_results/near_outputs/crim13_astar-near_010/test_results.json
Loading Baseline results from outputs/mice_results/near_outputs/crim13_astar-near_003/test_results.json
Loading Baseline results from outputs/mice_results/near_outputs/crim13_iddfs-near_020/test_results.json
Loading Baseline results from outputs/mice_results/near_outputs/crim13_enumeration_010/test_results.json
Loading Baseline results from outputs/mice_results/near_outputs/crim13_astar-near_011/test_results.json
Loading Baseline results from outputs/mice_results/near_outputs/crim13_enumeration_020/test_results.json
Loading Baseline results from outputs/mice_results/near_outputs/crim13_iddfs-near_003/test_results.json
Loading Baseline results from outputs/mice_results/near_output

## Load Neurosym-lib Results

In [ ]:
file_path = "outputs/bball_results/reproduction.pkl"
neurosym_results = {}
if Path(file_path).exists():
    with open(file_path, "rb") as f:
        results = pickle.load(f)
        file_name = Path(file_path).stem
        for i, result in enumerate(results):
            key = f"neurosym_{file_name}_{i:03d}"
            neurosym_results[key] = result
    print(f"  Loaded {len(results)} programs from {file_name}")
else:
    print(f"Warning: {file_path} not found")

  Loaded 40 programs from reproduction


## Create Comparison Table

In [ ]:
# Combine all results
all_results = {}
# all_results.update(baseline_results)
all_results.update(neurosym_results)

print(f"Total experiments: {len(all_results)}")

Total experiments: 51


In [48]:
# Create comparison dataframe
table_data = []

for name, result in all_results.items():
    # Extract metrics
    report = result.get("report", {})
    
    # Handle different report structures
    if isinstance(report, dict):
        if "report" in report:
            macro_avg = report["report"].get("macro avg", {})
            hamming_acc = report.get("hamming_accuracy", 0.0)
        elif "macro avg" in report:
            macro_avg = report["macro avg"]
            hamming_acc = report.get("hamming_accuracy", 0.0)
        else:
            # Need to compute metrics
            if "pred_vals" in result and "true_vals" in result:
                pred_vals = np.array(result["pred_vals"])
                true_vals = np.array(result["true_vals"])
                metrics = compute_metrics(pred_vals, true_vals)
                macro_avg = metrics["report"]["macro avg"]
                hamming_acc = metrics["hamming_accuracy"]
            else:
                continue
    else:
        continue
    
    row = {
        "experiment": name,
        "precision": macro_avg.get("precision", 0.0),
        "recall": macro_avg.get("recall", 0.0),
        "f1_score": macro_avg.get("f1-score", 0.0),
        "support": macro_avg.get("support", 0),
        "hamming_accuracy": hamming_acc,
        "time": result.get("time", 0.0),
    }
    table_data.append(row)

df = pd.DataFrame(table_data)

# Sort by f1_score descending
df = df.sort_values("hamming_accuracy", ascending=False)

print("\n" + "=" * 80)
print("RESULTS COMPARISON")
print("=" * 80)
print(df.to_string(index=False))
print("=" * 80)


RESULTS COMPARISON
               experiment  precision   recall  f1_score  support  hamming_accuracy         time
   crim13_enumeration_010   0.446849 0.500000  0.471933 295300.0          0.893698 81125.326581
    crim13_astar-near_001   0.446849 0.500000  0.471933 295300.0          0.893698  1563.009056
neurosym_reproduction_015   0.446849 0.500000  0.471933 295300.0          0.893698   150.186010
neurosym_reproduction_017   0.446849 0.500000  0.471933 295300.0          0.893698   150.186420
neurosym_reproduction_020   0.446849 0.500000  0.471933 295300.0          0.893698   239.672629
neurosym_reproduction_022   0.446849 0.500000  0.471933 295300.0          0.893698   239.673032
neurosym_reproduction_021   0.446849 0.500000  0.471933 295300.0          0.893698   239.672929
neurosym_reproduction_016   0.446849 0.500000  0.471933 295300.0          0.893698   150.186323
neurosym_reproduction_014   0.446849 0.500000  0.471933 295300.0          0.893698   129.036310
neurosym_reproductio

## Summary Statistics

In [ ]:
print("\nSUMMARY STATISTICS")
print("=" * 80)
print(f"Total experiments: {len(df)}")
print(f"\nBest F1-score: {df['f1_score'].max():.6f}")
print(f"  Experiment: {df.loc[df['f1_score'].idxmax(), 'experiment']}")
print(f"\nBest Hamming accuracy: {df['hamming_accuracy'].max():.6f}")
print(f"  Experiment: {df.loc[df['hamming_accuracy'].idxmax(), 'experiment']}")
print(f"\nAverage F1-score: {df['f1_score'].mean():.6f}")
print(f"Average Hamming accuracy: {df['hamming_accuracy'].mean():.6f}")
print(f"Average training time: {df['time'].mean():.2f} seconds")

# Baseline vs Neurosym comparison
baseline_df = df[df['experiment'].str.contains('baseline|crim13_astar|crim13_iddfs|enumeration', case=False, na=False)]
neurosym_df = df[df['experiment'].str.contains('neurosym|reproduction', case=False, na=False)]

if not baseline_df.empty:
    print(f"\nBaseline experiments: {len(baseline_df)}")
    print(f"  Avg F1: {baseline_df['f1_score'].mean():.6f}")
    print(f"  Avg Hamming Acc: {baseline_df['hamming_accuracy'].mean():.6f}")
    
if not neurosym_df.empty:
    print(f"\nNeurosym-lib experiments: {len(neurosym_df)}")
    print(f"  Avg F1: {neurosym_df['f1_score'].mean():.6f}")
    print(f"  Avg Hamming Acc: {neurosym_df['hamming_accuracy'].mean():.6f}")

# Reported program
reported_df = df[df['experiment'] == 'reported_program']
if not reported_df.empty:
    print("\nReported Program:")
    print(f"  F1: {reported_df['f1_score'].values[0]:.6f}")
    print(f"  Hamming Acc: {reported_df['hamming_accuracy'].values[0]:.6f}")
    print(f"  Time: {reported_df['time'].values[0]:.2f}s")

print("=" * 80)


SUMMARY STATISTICS
Total experiments: 51

Best F1-score: 0.530137
  Experiment: neurosym_reproduction_001

Best Hamming accuracy: 0.893698
  Experiment: crim13_enumeration_010

Average F1-score: 0.257404
Average Hamming accuracy: 0.397059
Average training time: 6036.53 seconds

Baseline experiments: 10
  Avg F1: 0.179051
  Avg Hamming Acc: 0.271186

Neurosym-lib experiments: 40
  Avg F1: 0.270592
  Avg Hamming Acc: 0.417227

Reported Program:
  F1: 0.513409
  Hamming Acc: 0.849062
  Time: 3.12s


## Save Results

In [ ]:
# Save CSV
csv_path = "outputs/bball_results/comparison.csv"
Path(csv_path).parent.mkdir(parents=True, exist_ok=True)
df.to_csv(csv_path, index=False)
print(f"Saved CSV to: {csv_path}")

# Save Markdown
md_path = "outputs/bball_results/comparison.md"
with open(md_path, "w") as f:
    f.write("# MICE-DSL NEAR Results Comparison\n\n")
    f.write(df.to_markdown(index=False))
    f.write("\n")
print(f"Saved Markdown to: {md_path}")

Saved CSV to: outputs/mice_results/comparison.csv
Saved Markdown to: outputs/mice_results/comparison.md


## Display Top Programs

In [52]:
# Show top 10 programs by F1 score
print("\nTop 10 Programs by F1-score:")
print("=" * 80)
top_10 = df.head(10)
print(top_10.to_string(index=False))

# Show program strings for top results if available
print("\nProgram Details:")
print("=" * 80)
for idx, row in top_10.iterrows():
    exp_name = row['experiment']
    if exp_name in all_results:
        result = all_results[exp_name]
        program_str = result.get('program_str', str(result.get('program', 'N/A')))
        print(f"\n{exp_name}:")
        print(f"  Program: {program_str}")
        print(f"  F1: {row['f1_score']:.6f}, Hamming Acc: {row['hamming_accuracy']:.6f}, Time: {row['time']:.2f}s")


Top 10 Programs by F1-score:
               experiment  precision   recall  f1_score  support  hamming_accuracy         time
   crim13_enumeration_010   0.446849 0.500000  0.471933 295300.0          0.893698 81125.326581
    crim13_astar-near_001   0.446849 0.500000  0.471933 295300.0          0.893698  1563.009056
neurosym_reproduction_015   0.446849 0.500000  0.471933 295300.0          0.893698   150.186010
neurosym_reproduction_017   0.446849 0.500000  0.471933 295300.0          0.893698   150.186420
neurosym_reproduction_020   0.446849 0.500000  0.471933 295300.0          0.893698   239.672629
neurosym_reproduction_022   0.446849 0.500000  0.471933 295300.0          0.893698   239.673032
neurosym_reproduction_021   0.446849 0.500000  0.471933 295300.0          0.893698   239.672929
neurosym_reproduction_016   0.446849 0.500000  0.471933 295300.0          0.893698   150.186323
neurosym_reproduction_014   0.446849 0.500000  0.471933 295300.0          0.893698   129.036310
neurosym_r